# Image classification example

In [1]:
import torch.nn
import torch.optim

import openml
import openml_pytorch
import openml_pytorch.layers
import openml_pytorch.config
import logging
import warnings

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore')

############################################################################
# Enable logging in order to observe the progress while running the example.
openml.config.logger.setLevel(logging.DEBUG)
openml_pytorch.config.logger.setLevel(logging.DEBUG)
############################################################################

############################################################################
import torch.nn as nn
import torch.nn.functional as F

In [2]:
# openml.config.apikey = 'key'

In [3]:
from openml_pytorch.trainer import OpenMLTrainerModule

trainer = OpenMLTrainerModule(
    file_dir=openml.config.get_cache_directory()+'/datasets/44312/PNU_Micro/images/',
    filename_col="FILE_NAME",
    target_mode="categorical",
    verbose = True,
)
openml_pytorch.config.trainer = trainer

In [4]:
# Download the OpenML task for the Meta_Album_PNU_Micro dataset.
task = openml.tasks.get_task(361987)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(13456, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 19) # To user - Remember to set correct size of last layer. 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net()

In [6]:
run = openml.runs.run_model_on_task(model, task, avoid_duplicate_runs=False)

Epochs: 100%|██████████| 3/3 [00:01<00:00,  1.55it/s]


In [7]:
run.predictions

,repeat,fold,sample,row_id,prediction,correct,confidence.HeadNeck,confidence.Pancreatic,confidence.Bile-duct,confidence.Thyroid,...,confidence.Cervix,confidence.Testis,confidence.Colon,confidence.Bladder,confidence.Liver,confidence.Stomach,confidence.Kidney,confidence.Skin,confidence.Breast,confidence.Lung
0,0,0,0,11,Kidney,Cervix,0.056092,0.050692,0.053833,0.055625,...,0.048228,0.050836,0.050543,0.055869,0.052432,0.053728,0.056920,0.053811,0.051691,0.055175
1,0,0,0,354,Kidney,Thyroid,0.056092,0.050692,0.053833,0.055625,...,0.048228,0.050836,0.050543,0.055869,0.052432,0.053728,0.056920,0.053811,0.051691,0.055175
2,0,0,0,612,Kidney,Lung,0.056092,0.050692,0.053833,0.055625,...,0.048228,0.050836,0.050543,0.055869,0.052432,0.053728,0.056920,0.053811,0.051691,0.055175
3,0,0,0,222,Kidney,Stomach,0.056092,0.050692,0.053833,0.055625,...,0.048228,0.050836,0.050543,0.055869,0.052432,0.053728,0.056920,0.053811,0.051691,0.055175
4,0,0,0,326,Kidney,Liver,0.056092,0.050692,0.053833,0.055625,...,0.048228,0.050836,0.050543,0.055869,0.052432,0.053728,0.056920,0.053811,0.051691,0.055175
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,0,9,0,431,Lung,HeadNeck,0.056626,0.050596,0.052565,0.052128,...,0.050651,0.052018,0.049877,0.053065,0.050705,0.052825,0.054239,0.055390,0.052028,0.056909
756,0,9,0,747,Lung,Pancreatic,0.056626,0.050596,0.052565,0.052128,...,0.050651,0.052018,0.049877,0.053065,0.050705,0.052825,0.054239,0.055390,0.052028,0.056909
757,0,9,0,591,Lung,Skin,0.056626,0.050596,0.052565,0.052128,...,0.050651,0.052018,0.049877,0.053065,0.050705,0.052825,0.054239,0.055390,0.052028,0.056909
758,0,9,0,193,Lung,Breast,0.056626,0.050596,0.052565,0.052128,...,0.050651,0.052018,0.049877,0.053065,0.050705,0.052825,0.054239,0.055390,0.052028,0.056909


# Pretrained Image classification example

In [10]:
import torch.nn
import torch.optim

import openml
import openml_pytorch
import openml_pytorch.layers
import openml_pytorch.config
import logging
import warnings

# Suppress FutureWarning messages
warnings.simplefilter(action='ignore')

############################################################################
# Enable logging in order to observe the progress while running the example.
openml.config.logger.setLevel(logging.DEBUG)
openml_pytorch.config.logger.setLevel(logging.DEBUG)
############################################################################

############################################################################
import torch.nn as nn
import torch.nn.functional as F
from openml_pytorch.trainer import OpenMLTrainerModule

In [11]:
# Example model. You can do better :)
import torchvision.models as models

# Load the pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Modify the last fully connected layer to the required number of classes
num_classes = 20
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, num_classes)

# Optional: If you're fine-tuning, you may want to freeze the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

# If you want to train the last layer only (the newly added layer)
for param in model.fc.parameters():
    param.requires_grad = True

In [12]:
from openml import OpenMLTask
def custom_optimizer_gen(model: torch.nn.Module, task: OpenMLTask) -> torch.optim.Optimizer:
    return torch.optim.Adam(model.fc.parameters())

trainer = OpenMLTrainerModule(
    file_dir=openml.config.get_cache_directory()+'/datasets/44312/PNU_Micro/images/',
    filename_col="FILE_NAME",
    target_mode="categorical",
    perform_validation = False,
    optimizer_gen = custom_optimizer_gen,

)
openml_pytorch.config.trainer = trainer

In [13]:
############################################################################
# Download the OpenML task for the Meta_Album_PNU_Micro dataset.
task = openml.tasks.get_task(361152)

############################################################################
# Run the model on the task (requires an API key).m
run = openml.runs.run_model_on_task(model, task, avoid_duplicate_runs=False)

Epochs: 100%|██████████| 3/3 [00:03<00:00,  1.21s/it]


In [16]:
run

OpenML Run
Uploader Name...................: None
Metric..........................: None
Local Result - Accuracy (+- STD): 0.0000 +- 0.0000
Run ID..........................: None
Task ID.........................: 361152
Task Type.......................: None
Task URL........................: https://www.openml.org/t/361152
Flow ID.........................: None
Flow Name.......................: torch.nn.ResNet.2f74ee0621acaf36
Flow URL........................: None
Setup ID........................: None
Setup String....................: Python_3.11.9. Torch_2.4.1. NumPy_1.26.4. SciPy_1.14.1. ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stri